In [1]:
!pip install opendatasets datasets accelerate --quiet
import opendatasets as od

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 21.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [2]:
od.download("https://www.kaggle.com/datasets/gondimalladeepesh/nvidia-documentation-question-and-answer-pairs")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: yousefriad
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/gondimalladeepesh/nvidia-documentation-question-and-answer-pairs


100%|██████████| 400k/400k [00:00<00:00, 62.5MB/s]

In [3]:
import pandas as pd
from datasets import Dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, TrainingArguments, Trainer
import torch
import re

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
data = pd.read_csv("/content/nvidia-documentation-question-and-answer-pairs/NvidiaDocumentationQandApairs.csv")[["question", "answer"]]
data.head()

,question,answer
0,What is Hybridizer?,Hybridizer is a compiler from Altimesh that en...
1,How does Hybridizer generate optimized code?,Hybridizer uses decorated symbols to express p...
2,What are some parallelization patterns mention...,The text mentions using parallelization patter...
3,How can you benefit from accelerators without ...,You can benefit from accelerators' compute hor...
4,What is an example of using Hybridizer?,An example in the text demonstrates using Para...


In [5]:
def clean_text(text):
  text = text.lower()
  text = re.sub('[^A-Za-z0-9]+', ' ', text)
  return text

In [6]:
test = "Is this REAL?! *$"
clean_text(test)

'is this real '

In [7]:
data["question"] = data["question"].apply(clean_text)
data["answer"] = data["answer"].apply(clean_text)

In [8]:
data.head()

,question,answer
0,what is hybridizer,hybridizer is a compiler from altimesh that en...
1,how does hybridizer generate optimized code,hybridizer uses decorated symbols to express p...
2,what are some parallelization patterns mention...,the text mentions using parallelization patter...
3,how can you benefit from accelerators without ...,you can benefit from accelerators compute hors...
4,what is an example of using hybridizer,an example in the text demonstrates using para...


In [9]:
train = data.sample(frac = 0.7)
test = data.drop(train.index)


val = test.sample(frac = 0.5)
test = test.drop(val.index)

In [10]:
print(f"Training shape {train.shape}")
print(f"Validation shape {val.shape}")
print(f"Testing shape {test.shape}")

Training shape (4976, 2)
Validation shape (1066, 2)
Testing shape (1066, 2)


In [11]:
original_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", torch_dtype = torch.float32)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [12]:
def tokenize_function(example):
  start_prompt = "According to the following question:\n\n"
  end_prompt = "\nAnswer:\n\n"

  prompt = [start_prompt + question + end_prompt for question in example["question"]]

  example["input_ids"] = tokenizer(prompt, padding = 'max_length', truncation = True, return_tensors = 'pt').input_ids
  example["labels"] = tokenizer(example["answer"], padding = 'max_length', truncation = True, return_tensors = 'pt').input_ids

  return example

In [13]:
train_data = Dataset.from_pandas(train)
train_tokenized_datasets = train_data.map(tokenize_function, batched = True)

Map:   0%|          | 0/4976 [00:00<?, ? examples/s]

In [14]:
train_tokenized_datasets

Dataset({
    features: ['question', 'answer', '__index_level_0__', 'input_ids', 'labels'],
    num_rows: 4976
})

we only need the labels and input_ids columns,  so we're going to drop the rest.

In [15]:
train_tokenized_datasets = train_tokenized_datasets.remove_columns(['question', 'answer', '__index_level_0__'])

In [16]:
train_tokenized_datasets

Dataset({
    features: ['input_ids', 'labels'],
    num_rows: 4976
})

In [17]:
val_data = Dataset.from_pandas(val)
val_tokenized_datasets = val_data.map(tokenize_function, batched = True)
val_tokenized_datasets = val_tokenized_datasets.remove_columns(['question', 'answer', '__index_level_0__'])

test_data = Dataset.from_pandas(test)
test_tokenized_datasets = test_data.map(tokenize_function, batched = True)
test_tokenized_datasets = test_tokenized_datasets.remove_columns(['question', 'answer', '__index_level_0__'])

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

Map:   0%|          | 0/1066 [00:00<?, ? examples/s]

In [18]:
EPOCHS = 5
LR = 1e-3
BATCH_SIZE = 2

traing_path = "./traing_nvidia_chatbot"

training_args = TrainingArguments(
    output_dir = traing_path,
    save_total_limit = 2,
    per_device_train_batch_size = BATCH_SIZE,
    per_device_eval_batch_size = BATCH_SIZE,
    learning_rate = LR,
    num_train_epochs = EPOCHS,
    evaluation_strategy = 'epoch'
)

trainer = Trainer(
    model = original_model,
    args = training_args,
    train_dataset = train_tokenized_datasets,
    eval_dataset = val_tokenized_datasets
)

trainer.train()

model_path = "./nvidia_chatbot_final_model"

trainer.model.save_pretrained(model_path)
#tokenizer.save_pretrained(model_path)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.187400,0.165287
2,0.137400,0.146795
3,0.092300,0.142510
4,0.059700,0.148836
5,0.036300,0.167121


we have 12440 **training instances** because:

**training data** = 4976


4976 / **BATCH_SIZE** = 4976 / 2 = 2488


2488 x **EPOCHS** = 2488 x 5 = 12440

In [19]:
eval_results = trainer.evaluate(eval_dataset = test_tokenized_datasets)

In [20]:
print(eval_results)

{'eval_loss': 0.17171137034893036, 'eval_runtime': 112.9619, 'eval_samples_per_second': 9.437, 'eval_steps_per_second': 4.718, 'epoch': 5.0}


In [21]:
test_text = "what is cuda nsight?"

start_prompt = "According to the following question:\n\n"
end_prompt = "\nAnswer:\n\n"

full_prompt = start_prompt + test_text + end_prompt
print(full_prompt)

According to the following question:

what is cuda nsight?
Answer:




In [22]:
from transformers import GenerationConfig

trained_model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_path)

tokenized_test_text = tokenizer(full_prompt, return_tensors = 'pt').input_ids.to(device)

model_output = trained_model.generate(tokenized_test_text,
                                      generation_config = GenerationConfig(max_new_tokens = 150))[0]

final_output = tokenizer.decode(model_output,
                                skip_special_tokens = True)

print(final_output)

OSError: Can't load tokenizer for './nvidia_chatbot_final_model'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure './nvidia_chatbot_final_model' is the correct path to a directory containing all relevant files for a T5TokenizerFast tokenizer.

Now let's compare between this model's output and the original model's output

In [ ]:
from transformers import GenerationConfig

original_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", torch_dtype = torch.float32).to(device)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

tokenized_test_text = tokenizer(full_prompt, return_tensors = 'pt').input_ids.to(device)

model_output = original_model.generate(tokenized_test_text,
                                       generation_config = GenerationConfig(max_new_tokens = 150))[0]

final_output = tokenizer.decode(model_output,
                                skip_special_tokens = True)

print(final_output)

Let's try another example

In [ ]:
test_data[53]

In [ ]:
test_text = "how does mig work with existing cuda programs?"

start_prompt = "According to the following question:\n\n"
end_prompt = "\nAnswer:\n\n"

full_prompt = start_prompt + test_text + end_prompt
print(full_prompt)

In [ ]:
from transformers import GenerationConfig

trained_model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_path)

tokenized_test_text = tokenizer(full_prompt, return_tensors = 'pt').input_ids.to(device)

model_output = trained_model.generate(tokenized_test_text,
                                      generation_config = GenerationConfig(max_new_tokens = 150))[0]

final_output = tokenizer.decode(model_output,
                                skip_special_tokens = True)

print(final_output)

In [ ]:
from transformers import GenerationConfig

original_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base", torch_dtype = torch.float32).to(device)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")

tokenized_test_text = tokenizer(full_prompt, return_tensors = 'pt').input_ids.to(device)

model_output = original_model.generate(tokenized_test_text,
                                       generation_config = GenerationConfig(max_new_tokens = 150))[0]

final_output = tokenizer.decode(model_output,
                                skip_special_tokens = True)

print(final_output)